# Reformat the mutation data

* Gives Mutation name and locations
* Mutation occurences in each sample 
* Consider mutation at each locus a mutation (doesn't matter the type)

In [43]:
file = open("/home/vungo/Downloads/gdac.broadinstitute.org_CHOL-TP.Mutation_CHASM.Level_4.2016012800.0.0/mutationslist.txt").read().split('\n')
print len(file)

5282


In [63]:
tag = "CHOL"
mutations = {}
samples = {}
for line in file:
    tmp = line.strip().split('\t')
    if len(tmp)==1:
        continue
    mut = tmp[1]+'_'+tmp[2]
    sample = tmp[0]
    if mut not in mutations:
        mutations[mut] = 1
    else:
        mutations[mut] += 1
    if sample not in samples:
        samples[sample] = {mut:1}
    else:
        samples[sample][mut] = 1
    

In [50]:
#mutations["chrX_1"]=1
translatedict = {}
for i in range(1,24):
    c = 'chr'+str(i)
    translatedict[c] =c 
#translatedict

In [58]:
#create a mutationname dict as well:
mutnames = {}

#make file SNP_locations:
target = open("/home/vungo/Downloads/SNP_locations.txt",'w')
target.write("snp\tchr\tpos")
mutationlocs = []
factor = 10**12
for mut in mutations:
    tmp = mut.split('_')
    try:
        loc = int(tmp[0].replace('chr',''))*factor + int(tmp[1])
    except:
        if 'x' in tmp[0] or 'X' in tmp[0]:
            loc = 25*factor + int(tmp[1]) #chrx is chr25
        elif 'y' in tmp[0] or 'Y' in tmp[0]:
            loc = 26*factor + int(tmp[1]) #chrx is chr26
        elif 'm' in tmp[0] or 'M' in tmp[0]:
            loc = 27*factor + int(tmp[1]) #chrx is chr27
    mutationlocs += [loc]
mutationlocs = sorted(mutationlocs)
for i in range(len(mutationlocs)):
    loc = mutationlocs[i]
    chromname = "chr"+str(loc/factor)
    try:
        chromname = translatedict[chromname]
    except:
        if chromname == 'chr25':
            chromname = 'chrX'
        elif chromname == 'chr26':
            chromname = 'chrY'
        elif chromname == 'chr27':
            chromname = 'chrM'
        else:
            print "ERROR"
            continue
    mutname = "snp_"+tag+"_"+str(i+1)
    mutnames[mutname] = chromname+'_'+str(loc%factor)
    line = mutname +'\t'+chromname+'\t'+str(loc%factor)
    
    #print line
    target.write('\n'+line)
target.close()

In [64]:
samplenames = sorted(samples.keys())
target = open("/home/vungo/Downloads/genotype.txt",'w')
line = "id"
for sam in samplenames:
    line += '\t'+sam
target.write(line)
for i in range(len(mutnames)):
    mutname = "snp_"+tag+"_"+str(i+1)
    vector = [0]*len(samples)
    mutlocname = mutnames[mutname]
    for i in range(len(samplenames)):
        sam = samplenames[i]
        if mutlocname in samples[sam]:
            vector[i] = 1
    line = mutname+'\t'+'\t'.join(map(str,vector))
    target.write('\n'+line)
    #print mutname
    #print vector
target.close()

In [61]:
map(str,[0, 0, 0, 0])

['0', '0', '0', '0']